In [1]:
from lightning.pytorch import Trainer, seed_everything
from lightning.pytorch.callbacks import ModelCheckpoint, StochasticWeightAveraging, EarlyStopping, LearningRateMonitor, LearningRateFinder

seed_everything(42, workers=True)

from scprint import scPrint
from scprint.trainer import TrainingMode
from scdataloader import DataModule 
import pandas as pd
from scdataloader.utils import load_genes
import lamindb as ln
import pandas as pd
import numpy as np
from scdataloader import DataModule, Preprocessor, utils
from scdataloader.preprocess import additional_postprocess, additional_preprocess

import os
import urllib.request
import torch

from scprint import scPrint
import scanpy as sc

import torch
torch.set_float32_matmul_precision('medium')

%load_ext autoreload
%autoreload 2

Seed set to 42


→ connected lamindb: anonymous/test2


In [2]:
ckpt_path = "large.ckpt"
if not os.path.exists(ckpt_path):
    url = "https://huggingface.co/jkobject/scPRINT/resolve/main/large.ckpt"
    urllib.request.urlretrieve(url, ckpt_path)

In [3]:
model = scPrint.load_from_checkpoint(
        ckpt_path,
        precpt_gene_emb=None,
        # triton gets installed so it must think it has cuda enabled
        transformer="normal", #else normal, without flashattention
)

changing the number of classes, could lead to issues
RuntimeError caught: scPrint is not attached to a `Trainer`.


In [5]:
adata= sc.read_h5ad("adata_spatial_raw.h5ad", backed="r")

In [28]:
adata

AnnData object with n_obs × n_vars = 324122 × 16927 backed at 'adata_spatial_raw.h5ad'
    obs: 'in_tissue', 'array_row', 'array_col', 'batch'
    obsm: 'spatial'

In [6]:
adata = adata[:10000].to_memory()

In [7]:
adata.obs['organism_ontology_term_id'] = "NCBITaxon:9606"
adata.obs_names_make_unique()
preprocessor = Preprocessor(is_symbol=True, force_preprocess=True, skip_validate=True, do_postp=False, use_raw=False)
adata = preprocessor(adata.copy())
adata

Dropping layers:  KeysView(Layers with keys: )
checking raw counts
removed 0 non primary cells, 10000 renamining
filtered out 12 cells, 9988 renamining
Removed 0 genes.
startin QC
Seeing 2113 outliers (21.16% of total dataset):
done
AnnData object with n_obs × n_vars = 9988 × 70611
    obs: 'in_tissue', 'array_row', 'array_col', 'batch', 'organism_ontology_term_id', 'nnz', 'n_genes', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_20_genes', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt', 'total_counts_ribo', 'log1p_total_counts_ribo', 'pct_counts_ribo', 'total_counts_hb', 'log1p_total_counts_hb', 'pct_counts_hb', 'outlier', 'mt_outlier'
    var: 'uid', 'symbol', 'ncbi_gene_ids', 'biotype', 'synonyms', 'description', 'organism_id', 'mt', 'ribo', 'hb', 'organism', 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts'
    uns: 'unseen_genes'
    obsm: 'sp

AnnData object with n_obs × n_vars = 9988 × 70611
    obs: 'in_tissue', 'array_row', 'array_col', 'batch', 'organism_ontology_term_id', 'nnz', 'n_genes', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_20_genes', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt', 'total_counts_ribo', 'log1p_total_counts_ribo', 'pct_counts_ribo', 'total_counts_hb', 'log1p_total_counts_hb', 'pct_counts_hb', 'outlier', 'mt_outlier'
    var: 'uid', 'symbol', 'ncbi_gene_ids', 'biotype', 'synonyms', 'description', 'organism_id', 'mt', 'ribo', 'hb', 'organism', 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts'
    uns: 'unseen_genes'
    obsm: 'spatial'

In [8]:
from scdataloader.data import SimpleAnnDataset
from torch.utils.data import DataLoader
from scdataloader import Collator, Preprocessor
from sklearn.model_selection import train_test_split


In [9]:
train_idx, test_idx = train_test_split(range(adata.n_obs), test_size=0.2, random_state=42)

In [10]:
adataset_train = SimpleAnnDataset(
    adata[train_idx], obs_to_output=["organism_ontology_term_id"]
)
col = Collator(
    organisms=["NCBITaxon:9606"],
    how="random expr",
    max_len=10_000,
)
train_dataloader = DataLoader(
    adataset_train,
    collate_fn=col,
    batch_size=2,
    num_workers=8,
    shuffle=True,
)

adataset_test = SimpleAnnDataset(
    adata[test_idx], obs_to_output=["organism_ontology_term_id"]
)
test_dataloader = DataLoader(
    adataset_test,
    collate_fn=col,
    batch_size=4,
    num_workers=8,
    shuffle=True,
)

In [11]:
from lightning.pytorch.callbacks.finetuning import BaseFinetuning
import torch.nn as nn
from torch.optim import Optimizer
import torch.optim as optim

In [12]:
class FineTuneMode(BaseFinetuning):
    def __init__(self, lr: float = 1e-5, optimizer: str = "adamW"):
        super().__init__()
        self.lr = lr
        if optimizer == "adamW":
            self.optimizer = optim.AdamW
        elif optimizer == "adam":
            self.optimizer = optim.Adam
        else:
            raise ValueError(f"Optimizer {optimizer} not supported")

    def freeze_before_training(self, pl_module: "pl.LightningModule") -> None:
        # Freeze all parameters in the model
        self.freeze(pl_module, train_bn=False)

    def finetune_function(self, pl_module: "pl.LightningModule", epoch: int, optimizer: Optimizer) -> None:
        # Get all modules in a flat list
        modules = list(pl_module.modules())
        # Get only the last 2 layers that are nn.Module instances
        last_layers = [
            m
            for m in modules
            if isinstance(m, nn.Module) and len(list(m.parameters(recurse=False))) > 0
        ][-2:]

        # Unfreeze the last 2 layers and add them to optimizer with lower learning rate
        self.unfreeze_and_add_param_group(
            modules=last_layers, optimizer=optimizer, lr=self.lr, train_bn=False
        )

In [13]:
finetuner = FineTuneMode()

In [21]:
i['x'].min

torch.float32

In [17]:
for i in train_dataloader:
    print(i)
    break

{'x': tensor([[5., 4., 1.,  ..., 0., 0., 0.],
        [1., 1., 1.,  ..., 0., 0., 0.]]), 'genes': tensor([[11703, 37479, 21935,  ..., 46437, 49565, 62810],
        [ 6928, 45062,  9702,  ..., 45707, 35559, 49839]], dtype=torch.int32), 'class': tensor([], size=(2, 0), dtype=torch.int32), 'tp': tensor([0., 0.]), 'depth': tensor([38423., 21201.])}


In [14]:
trainer = Trainer(gradient_clip_val=100, max_time={"hours": 2}, callbacks=[
                  finetuner], accumulate_grad_batches=1, check_val_every_n_epoch=1, logger=None, num_sanity_val_steps=0)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [16]:
trainer.fit(model, train_dataloaders=train_dataloader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name                            | Type                         | Params | Mode 
-----------------------------------------------------------------------------------------
0 | gene_encoder                    | GeneEncoder                  | 22.9 M | train
1 | expr_encoder                    | ContinuousValueEncoder       | 264 K  | train
2 | pos_encoder                     | PositionalEncoding           | 0      | train
3 | class_encoder                   | CategoryValueEncoder         | 3.6 K  | train
4 | depth_encoder                   | ContinuousValueEncoder       | 264 K  | train
5 | transformer                     | FlashTransformer             | 50.4 M | train
6 | expr_decoder                    | ExprDecoder                  | 528 K  | train
7 | cls_decoders                    | ModuleDict                   | 1.9 M  | train
8 | grad_reverse_discriminator_loss | AdversarialDiscriminatorLoss | 822 K  | train
9 | mvc_decoder            

Training: |          | 0/? [00:00<?, ?it/s]

> /home/ec2-user/SageMaker/scPRINT/scprint/model/model.py(476)_encoder()
    474             elif self.normalization == "log":
    475                 import pdb; pdb.set_trace()
--> 476                 norm_expr = torch.log2(1 + expression)
    477             else:
    478                 raise ValueError(f"Unknown normalization: {self.normalization}")



ipdb>  expression


*** RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


ipdb>  q


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
